# Vehicle Detection(detection)

In [1]:
import numpy as np
import cv2
import imageio
from sklearn.externals import joblib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import vehicleDetectUtil as vehicleUtil
import vehicleDetect_svmVar as svmVar

In [3]:
svc = joblib.load('svm.pkl')
X_scaler = joblib.load('svm_scaler.pkl')
pca = joblib.load('svm_pca.pkl')

from scipy.ndimage.measurements import label
windowSizes = [96, 128, 145]
windowOverlap = 0.75
classifier_imgSize = 64
img = cv2.imread('test_images/test1.jpg')
img_size = img.shape
x_start_stop = [0, img_size[1]]
y_start_stop = [int(img_size[0]/2), img_size[0]-32]
windows = vehicleUtil.slide_window(img, x_start_stop=[None, None], y_start_stop=y_start_stop, windowSizeAr=windowSizes, xy_overlap=(windowOverlap, windowOverlap))

In [4]:
filename = 'project_video.mp4'
reader = imageio.get_reader(filename, 'ffmpeg')

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('output.avi',fourcc, 25.0, (1280,720))

for index, img in enumerate(reader):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    heat = np.zeros_like(img[:,:,0]).astype(np.float)
    #print('SVM: extracting windows...')
    imgs = vehicleUtil.get_window_imgs(img, windows, classifier_imgSize)
    features = vehicleUtil.extract_features(imgs, readImg=False, cspace=svmVar.spatial_clr, spatial_size=(svmVar.spatial, svmVar.spatial),
                            hist_bins=svmVar.histbin, hist_range=(0, 256), spatialFeat = svmVar.spatialFeat, histFeat = svmVar.histFeat,
                            hogFeat=svmVar.hogFeat, hog_cspace=svmVar.hog_clrspace, hog_orient=svmVar.orient, hog_pix_per_cell=svmVar.pix_per_cell, hog_cell_per_block=svmVar.cell_per_block, hog_channel=svmVar.hog_channel)
    X = np.vstack((features)).astype(np.float64)
    scaled_X = X_scaler.transform(X)
    scaled_X = pca.transform(scaled_X)
    pred_bin = svc.predict(scaled_X[:])
    #print('plotting hot windows...')
    
    ind = [x for x in range(len(pred_bin)) if pred_bin[x]==1]
    hot_windows = [windows[i] for i in ind]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    window_img = vehicleUtil.draw_boxes(img, hot_windows, color=(0, 0, 255), thick=6)

    heat = vehicleUtil.add_heat(heat,hot_windows)
    heatmap = np.clip(heat, 0, 255)
    labels = label(heatmap)
    label_img = vehicleUtil.draw_labeled_bboxes(np.copy(img), labels)
    
    out.write(cv2.cvtColor(label_img, cv2.COLOR_RGB2BGR))
    #plt.imshow(label_img)
    #plt.show()
    
out.release()
print('finished')

/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


finished
